In [1]:
%config Completer.use_jedi = False
import sys
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import scipy.io
import time
import threading
import gc
import copy

sys.path.append("/end/home/rh2310/morpho_repo/turing_codebase")
from turing.utils import *
import turing.pinns as tu
from turing.loss_functions import *

GPU = True
import os

if GPU:
    txt_device = 'gpu:0'
else:
    txt_device = 'cpu:0'    
    os.environ["CUDA_VISIBLE_DEVICES"]="-1"

2022-11-16 09:34:51.930170: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2022-11-16 09:35:05.356398: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2022-11-16 09:35:05.373921: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:2d:00.0 name: Quadro RTX 6000 computeCapability: 7.5
coreClock: 1.77GHz coreCount: 72 deviceMemorySize: 23.65GiB deviceMemoryBandwidth: 625.94GiB/s
2022-11-16 09:35:05.373987: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2022-11-16 09:35:05.378552: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcublas.so.10
2022-11-16 09:35:05.382037: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcufft.so.10

# Load data from the output of Brusselator model

In [2]:
ds = TINN_Dataset(tf.float32,
                  np.array([1, 2, 3, 4]), 
                  np.array([1, 2, 3, 4]),
                  np.array([1, 2, 3, 4, 5, 6, 7]), 
                  np.array([1, 2, 3, 4]),
                  np.array([1, 2, 3, 4, 5, 6, 7, 8]))

In [3]:
list(ds.batch(3).as_numpy_iterator())

[(array([1., 2., 3.], dtype=float32),
  array([1., 2., 3.], dtype=float32),
  array([1., 2., 3.], dtype=float32),
  array([1., 2., 3.], dtype=float32),
  array([1., 2., 3.], dtype=float32)),
 (array([4., 1., 2.], dtype=float32),
  array([4., 1., 2.], dtype=float32),
  array([4., 5., 6.], dtype=float32),
  array([4., 1., 2.], dtype=float32),
  array([4., 5., 6.], dtype=float32)),
 (array([3., 4.], dtype=float32),
  array([3., 4.], dtype=float32),
  array([7., 1.], dtype=float32),
  array([3., 4.], dtype=float32),
  array([7., 8.], dtype=float32))]

In [4]:
len(())

0

In [5]:
ds.save(".", "ds")

In [6]:
ds2 = TINN_Dataset.restore(".", "ds")

In [7]:
ds2 = TINN_Dataset.restore(".", "ds")

In [8]:
ds2.sizes

array([4, 4, 7, 4, 8])

In [9]:
list(ds2.batch(3).as_numpy_iterator())

[(array([1., 1., 1.]),
  array([2., 2., 2.]),
  array([3., 3., 3.]),
  array([4., 4., 4.]),
  array([1., 1., 5.]),
  array([2., 2., 6.]),
  array([3., 3., 7.]),
  array([4., 4., 1.])),
 (array([1., 1.]),
  array([2., 2.]),
  array([3., 3.]),
  array([4., 4.]),
  array([1., 5.]),
  array([2., 6.]),
  array([3., 7.]),
  array([4., 8.]))]

In [10]:
list(indices(2, False, 9, 2))

[[array([0, 1]), array([], dtype=int64)],
 [array([2, 3]), array([], dtype=int64)],
 [array([4, 5]), array([], dtype=int64)],
 [array([6, 7]), array([], dtype=int64)],
 [array([8]), array([0, 1])]]

In [11]:
ds_original = TINN_Single_Sim_Dataset(str(os.path.abspath("../../test/outputs_Brusselator_100_by_100/Brusselator_1"),),
                                      "Brusselator_1",
                                      thining_start=-2,
                                      signal_to_noise=0)

In [28]:
ds_original = TINN_Single_Sim_Dataset(str(os.path.abspath("../../test/outputs_Brusselator_100_by_100/Brusselator_1"),),
                                      "Brusselator_1",
                                      thining_start=-2,
                                      signal_to_noise=0)

In [12]:
ds_original.save(".", "ds_ds")

In [13]:
ds2 = TINN_Dataset.restore(".", "ds_ds")

In [14]:
ds2.simulation

Simulation(name='Brusselator_1', n=(100, 100), L=(2, 2), Ds=(0.002, 0.04), dt=0.0002, t_start=0, t_end=512.0, t_steps=516, dt_arr=None, parameters={'A': 2, 'B': 3}, steady_state_func=None, perturbation_size=0.1, kinetic_func=None, tol=0.001, sample_parameters=True, sample_parameters_num=10, sample_parameters_std=(0.2, 0.2), same_init=True, c0=None)

In [2]:
dataset = TINN_Single_Sim_Dataset.restore(
    os.path.abspath("../../test/outputs_tinn_Brusselator_100_by_100_noise/Brusselator_100_noise"),
                                 "Brusselator_100_noise_dataset")

In [3]:
dataset.__parameters__

{'lb': array([  0.        ,   0.        , 511.00582524]),
 'ub': array([  2.,   2., 512.]),
 'simulation': Simulation(name='Brusselator_1', n=(100, 100), L=(2, 2), Ds=(0.002, 0.04), dt=0.0002, t_start=0, t_end=512.0, t_steps=516, dt_arr=None, parameters={'A': 2, 'B': 3}, steady_state_func=None, perturbation_size=0.1, kinetic_func=None, tol=0.001, sample_parameters=True, sample_parameters_num=10, sample_parameters_std=(0.2, 0.2), same_init=True, c0=None),
 'ts': array([511.00582524, 512.        ]),
 'x_size': 20000,
 'x_pde_size': 20000}

In [4]:
dataset.ts

array([511.00582524, 512.        ])

In [54]:
dataset.__parameters__["x_size"] = ds_original.x_size
dataset.__parameters__["x_pde_size"] = ds_original.x_pde_size
dataset.__parameters__

{'lb': array([  0.        ,   0.        , 511.00582524]),
 'ub': array([  2.,   2., 512.]),
 'simulation': Simulation(name='Brusselator_1', n=(100, 100), L=(2, 2), Ds=(0.002, 0.04), dt=0.0002, t_start=0, t_end=512.0, t_steps=516, dt_arr=None, parameters={'A': 2, 'B': 3}, steady_state_func=None, perturbation_size=0.1, kinetic_func=None, tol=0.001, sample_parameters=True, sample_parameters_num=10, sample_parameters_std=(0.2, 0.2), same_init=True, c0=None),
 'ts': array([511.00582524, 512.        ]),
 'x_size': 20000,
 'x_pde_size': 20000}

In [55]:
dataset.save(os.path.abspath("../../test/outputs_tinn_Brusselator_100_by_100_noise/Brusselator_100_noise"), 
             "Brusselator_100_noise_dataset")

In [5]:
dataset2 = TINN_Single_Sim_Dataset.restore(os.path.abspath("."),
                                 "test_ds")

In [6]:
dataset2.__parameters__

{'lb': array([  0.        ,   0.        , 511.00582524]),
 'ub': array([  2.,   2., 512.]),
 'simulation': Simulation(name='Brusselator_1', n=(100, 100), L=(2, 2), Ds=(0.002, 0.04), dt=0.0002, t_start=0, t_end=512.0, t_steps=516, dt_arr=None, parameters={'A': 2, 'B': 3}, steady_state_func=None, perturbation_size=0.1, kinetic_func=None, tol=0.001, sample_parameters=True, sample_parameters_num=10, sample_parameters_std=(0.2, 0.2), same_init=True, c0=None),
 'ts': array([511.00582524, 512.        ]),
 'x_size': 20000,
 'x_pde_size': 20000}

In [2]:
import os
data_path = os.path.abspath("../turing_Brusselator.npy")
with open(data_path, 'rb') as f:
    data = np.load(f)
    
data_path = os.path.abspath("../turing_t_Brusselator.npy")
with open(data_path, 'rb') as f:
    t_star = np.load(f) 
    
# Thining the dataset
t_star = t_star[20::16]
data = data[..., 20::16]

T = t_star.shape[0]    
    
L = 1.0
x_size = data.shape[1]
y_size = data.shape[2]
N = x_size*y_size

model_params = {'training_data_size': T*N,#T*32,
                'pde_data_size': (T*N)//(32),
                'boundary_data_size':((x_size + y_size)*T),
                'with_neighbors': True,
                'shuffle':True}#//(8)}

dataset = create_dataset(data, t_star, N, T, L, **model_params)
lb = dataset['lb']
ub = dataset['ub']
obs_X = dataset['obs_input']
obs_Y = dataset['obs_output']
pde_X = dataset['pde']
pbc_X = np.concatenate([dataset['boundary_LB'], dataset['boundary_RT']], axis=1)
#boundary_LB = dataset['boundary_LB']
#boundary_RT = dataset['boundary_RT']

# diff_X_left = dataset['X_left']
# diff_X_right = dataset['X_right']
# diff_Y_up = dataset['Y_up']
# diff_Y_bottom = dataset['Y_bottom']
diff_X = np.concatenate([obs_X,
                         dataset['X_left'][:, np.newaxis], 
                         dataset['X_right'][:, np.newaxis], 
                         dataset['Y_up'][:, np.newaxis], 
                         dataset['Y_bottom'][:, np.newaxis]], axis=1)

In [3]:
obs_X.shape

(15360, 3)

In [4]:
ds = TINN_Dataset(tf.float64, 
                  obs_X, obs_Y, pde_X, 
                  pbc_X,
                  diff_X)

In [27]:
a = list(ds.batch(2).take(1).as_numpy_iterator())

In [31]:
a[0][3]

array([[ 0.        ,  0.83870968, 14.11764706,  1.        ,  0.83870968,
        14.11764706],
       [ 0.48387097,  0.        , 95.68627451,  0.48387097,  1.        ,
        95.68627451]])

# Run Koch-Meinhardt's ASDM model with given diffusion constants and correct space-time scalse to find other parameters

In [5]:
#layers = [3, 64, 64, 64, 64, 2]
#layers = [3, 128, 128, 128, 128, 2]
layers = [3, 10, 10, 10, 2]

pinn = tu.NN(layers, lb, ub, dtype=tf.float64)
pde_loss = Koch_Meinhard(D_u=PDE_Parameter("D_u", Parameter_Type.CONSTANT, 1,dtype=pinn.dtype),
                D_v=PDE_Parameter("D_v", Parameter_Type.CONSTANT, 1,dtype=pinn.dtype),
                sigma_u=PDE_Parameter("sigma_u", Parameter_Type.CONSTANT, 1,dtype=pinn.dtype),
                sigma_v=PDE_Parameter("sigma_v", Parameter_Type.CONSTANT, 1,dtype=pinn.dtype),
                rho_u=PDE_Parameter("rho_u", Parameter_Type.CONSTANT, 1,dtype=pinn.dtype),
                rho_v=PDE_Parameter("rho_v", Parameter_Type.CONSTANT, 1,dtype=pinn.dtype),
                mu_u=PDE_Parameter("mu_u", Parameter_Type.CONSTANT, 1,dtype=pinn.dtype),
                kappa_u=PDE_Parameter("kappa_u", Parameter_Type.CONSTANT, 1,dtype=pinn.dtype),
                )
#pde_non_zero = Non_zero_params("ASDM", parameters=[pde_loss.D_u, pde_loss.D_v], print_precision=".10f")

l2_norm = L2()
l_inf_norm = L_Inf()

In [6]:
class dummy_extra_loss(PDE_Residual):
    def __init__(
        self,
        print_precision=".5f",
    ):
        """
        """

        super().__init__(name="dummy_extra_loss", print_precision=print_precision)
        self._trainables_ = ()
        self.AAA = tf.Variable(1.0, dtype=tf.float64, name="AAA")
        #self._trainables_ += (self.AAA,)

    @tf.function
    def residual(self, pinn, x):

        
        return self.AAA * tf.reduce_sum(x)

In [13]:
class Periodic_Boundary_Condition(PDE_Residual):
    def __init__(
        self,
        print_precision=".5f",
    ):
        """
        """

        super().__init__(name="Periodic_Boundary_Condition", print_precision=print_precision)

    @tf.function
    def residual(self, pinn, x):
        print(self.name)
        print(x)
        y1 = pinn.net(x[:,:3])
        y2 = pinn.net(x[:,3:])
        return tf.reduce_mean(tf.abs(y1 - y2))

In [14]:
pbc_loss = Periodic_Boundary_Condition()

In [49]:
class Diffusion_Loss(PDE_Residual):
    def __init__(
        self,
        print_precision=".5f",
    ):
        """
        """

        super().__init__(name="Diffusion_Loss", print_precision=print_precision)

    @tf.function
    def residual(self, pinn, x):
        print(self.name)
        print(x)
        
        val = pinn.net(x[:, :3])
        x_left = pinn.net(tf.concat([x[:, 3:4], x[:, 1:3]], axis=1) )
        x_right = pinn.net(tf.concat([x[:, 4:5], x[:, 1:3]], axis=1))
        y_up = pinn.net(tf.concat([x[:, 0:1], x[:, 5:6], x[:, 2:3]], axis=1))
        y_bottom = pinn.net(tf.concat([x[:, 0:1], x[:, 6:7], x[:, 2:3]], axis=1))
        
        return tf.reduce_sum(x_left + x_right + y_up + y_bottom - 4.0 *val)

In [50]:
diff_loss = Diffusion_Loss()

In [51]:
model = tu.TINN(pinn, 
                pde_loss, 
                loss = l2_norm + l_inf_norm,
                extra_loss = [pbc_loss, diff_loss],
                #non_zero_loss=pde_non_zero, 
                #optimizer=keras.optimizers.Adam(learning_rate=0.0001), 
                alpha = 0.5,
                print_precision=".10f")

In [52]:
results = model.train(epochs=10,
                      batch_size=1000,#dataset.x//8,
                      dataset=ds,
                      print_interval=1,
                      stop_threshold=1e-5,
                      sample_losses=True, 
                      sample_parameters=True,
                      sample_regularisations=True,
                      sample_gradients=True,
                      regularise=True
                     )


Start of epoch 0
Diffusion_Loss
Tensor("x:0", shape=(1000, 7), dtype=float64)
Diffusion_Loss
Tensor("x:0", shape=(360, 7), dtype=float64)
Training observations acc over epoch: 12.8255968094
total loss: 230.7637145786, total regularisd loss (sum of batches): 914.7846136935
obs u loss: 43.7807871108, obs v loss: 7.5215979153
pde u loss: 177.1006628888, pde v loss: 1.9698397232
lambda obs u: 1.0000000000, lambda obs v: 1.0000000000
lambda pde u: 1.0000000000, lambda pde v: 1.0000000000

extra loss Periodic_Boundary_Condition: 0.3908269404
extra loss Diffusion_Loss: 0.3908269404
Time taken: 11.21s

Start of epoch 1
Training observations acc over epoch: 12.2076787949
total loss: 186.6047953265, total regularisd loss (sum of batches): 853.7531068693
obs u loss: 42.5074216740, obs v loss: 6.3232963852
pde u loss: 135.3811713387, pde v loss: 2.0752015749
lambda obs u: 2.3970821524, lambda obs v: 0.5834799033
lambda pde u: 0.5193570697, lambda pde v: 0.5000808747

extra loss Periodic_Boundary_

In [11]:
model.save(".", "test_model")

In [7]:
model2 = tu.TINN.restore(".", "test_model", ds)

2022-11-13 17:59:40.059728: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
2022-11-13 17:59:41.111141: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcublas.so.10


In [8]:
results2 = model2.train(epochs=10,
                      batch_size=1000,#dataset.x//8,
                      dataset=ds,
                      print_interval=1,
                      stop_threshold=1e-5,
                      sample_losses=True, 
                      sample_parameters=True,
                      sample_regularisations=True,
                      sample_gradients=True,
                      regularise=True
                     )


Start of epoch 0
Training observations acc over epoch: 11.4252824783
total loss: 2530.0493930381, total regularisd loss (sum of batches): 40937.2246997311
obs u loss: 40.5163734476, obs v loss: 5.1847551765
pde u loss: 21.7140385971, pde v loss: 0.1342258168
lambda obs u: 3.9034932133, lambda obs v: 0.0914803663
lambda pde u: 0.0028442035, lambda pde v: 0.0021822170

extra loss dummy_extra_loss: 2462.5000000000
Time taken: 17.40s

Start of epoch 1
Training observations acc over epoch: 11.0286703110
total loss: 2537.5991540953, total regularisd loss (sum of batches): 40855.9875441558
obs u loss: 39.2155793816, obs v loss: 4.8991029926
pde u loss: 30.9466357943, pde v loss: 0.0378359268
lambda obs u: 3.9120081729, lambda obs v: 0.0831288507
lambda pde u: 0.0037717003, lambda pde v: 0.0010912761

extra loss dummy_extra_loss: 2462.5000000000
Time taken: 2.95s

Start of epoch 2
Training observations acc over epoch: 10.6763000488
total loss: 2546.0868373962, total regularisd loss (sum of ba

In [4]:
model = tu.TINN.restore(".", "ttt_model", dataset)

2022-11-01 10:38:41.746544: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


In [5]:
results = model.train(epochs=10,
                      batch_size=dataset.x_size//8,
                      dataset=dataset,
                      print_interval=1,
                      stop_threshold=1e-5,
                      sample_losses=True, 
                      sample_parameters=True,
                      sample_regularisations=True,
                      sample_gradients=True,
                      regularise=True
                     )


Start of epoch 0
Training observations acc over epoch: 3.6630735397
total loss: 16.3038350046, total regularisd loss (sum of batches): 350.4495756340
obs u loss: 11.8244021490, obs v loss: 2.8278921080
pde u loss: 0.6747090917, pde v loss: 0.9768316559
lambda obs u: 3.6311555915, lambda obs v: 0.2805856632
lambda pde u: 0.0321953584, lambda pde v: 0.0560633869

Time taken: 16.21s

Start of epoch 1
Training observations acc over epoch: 3.5662655830
total loss: 15.8309160803, total regularisd loss (sum of batches): 339.7509331592
obs u loss: 11.4584784700, obs v loss: 2.8065849738
pde u loss: 0.5834595862, pde v loss: 0.9823930503
lambda obs u: 3.6171997357, lambda obs v: 0.2971298701
lambda pde u: 0.0251147548, lambda pde v: 0.0605556393

Time taken: 5.03s

Start of epoch 2
Training observations acc over epoch: 3.4627938271
total loss: 15.3313658305, total regularisd loss (sum of batches): 327.5783402234
obs u loss: 11.0719076388, obs v loss: 2.7792672389
pde u loss: 0.4922586038, pde 

In [15]:
weight_values = model.optimizer.get_weights()

In [16]:
len(weight_values)

33

In [76]:
data_path = pathlib.PurePath(os.path.abspath("../../test/outputs_tinn_Brusselator_100_by_100_noise/Brusselator_000_noise/Brusselator_000_noise_model_optimizer.pkl"))
with open(data_path, "rb") as f:
    data = np.load(f, allow_pickle=True)

In [115]:
len(data)

25

In [114]:
len(weight_values)

49

In [92]:
with open(os.path.abspath("./tttt.npy"), "wb") as f:
    pickle.dump(weight_values, f)

In [93]:
with open(os.path.abspath("./tttt.npy"), "rb") as f:
    weight_values2 = pickle.load(f)

In [112]:
weight_values[7].shape

(1, 128)

In [113]:
weight_values2[7].shape

(1, 128)

In [6]:
model.save(".", "ttt_model")

In [9]:
model.optimizer.from_config(model.optimizer.get_config())

In [31]:
import pathlib
path = pathlib.PurePath(".").joinpath("ttt_model")
# asset = tf.saved_model.load(str(path))
with open(f"{str(path)}.pkl", "rb") as f:
    model2 = pickle.load(f)
with open(f"{str(path)}_optimizer.pkl", "rb") as f:
    weight_values2 = pickle.load(f)
with open(f"{str(path)}_optimizer_config.pkl", "rb") as f:
    conf2 = pickle.load(f)


In [32]:
 model2.optimizer = keras.optimizers.Adam(learning_rate=5e-4).from_config(conf2)

In [25]:
trainables = model2.pinn.trainable_variables + model2.pde_residual.trainables()
for extra_loss in model2.extra_loss:
    trainables += extra_loss.trainables()

if model2.non_zero_loss is not None:
    non_zero_params = model2.non_zero_loss.parameters
# dummy zero gradients
zero_grads = [tf.zeros_like(w) for w in trainables]
# save current state of variables
saved_vars = [tf.identity(w) for w in trainables]



if model2.non_zero_loss is not None:
    # dummy zero gradients
    zero_grads_params = [tf.zeros_like(w) for w in model2.non_zero_loss.parameters]
    # save current state of variables
    saved_vars_params = [tf.identity(w) for w in model2.non_zero_loss.parameters]

# Apply gradients which don't do nothing with Adam
model2.optimizer.apply_gradients(zip(zero_grads, trainables))
if model2.non_zero_loss is not None:
    model2.optimizer.apply_gradients(zip(zero_grads_params, model2.non_zero_loss.parameters))

# Reload variables
[x.assign(y) for x, y in zip(trainables, saved_vars)]
if model2.non_zero_loss is not None:
    [x.assign(y) for x, y in zip(model2.non_zero_loss.parameters, saved_vars_params)]

In [33]:
len(trainables)

8

In [34]:
len(weight_values2)

33

In [44]:
X, Y = next(iter(dataset.take(1)))#).numpy()

In [84]:
x_size = 5
y_size = 6
x_domain =  np.linspace(0,x_size-1 , x_size)
y_domain =  np.linspace(0, y_size-1, y_size)

X, Y = np.meshgrid(x_domain, y_domain, sparse=False, indexing="ij")
XX = np.tile(X.flatten(), T)  # N x T
YY = np.tile(Y.flatten(), T)  # N x T
TT = np.repeat(t_star[-T:], N)  # T x N

In [96]:
np.r_[y_domain[1:], y_domain[0]]

array([1., 2., 3., 4., 5., 0.])

In [85]:
x_domain

array([0., 1., 2., 3., 4.])

In [128]:
X2, Y2 = np.meshgrid(np.r_[x_domain[-1], x_domain[:-1]],
                             np.r_[y_domain[-1], y_domain[:-1]],
                             sparse=False, indexing="ij")
XX2 = np.tile(X2.flatten(), T)  # N x T
YY2 = np.tile(Y2.flatten(), T)  # N x T

In [129]:
X1

array([[1., 1., 1., 1., 1., 1.],
       [2., 2., 2., 2., 2., 2.],
       [3., 3., 3., 3., 3., 3.],
       [4., 4., 4., 4., 4., 4.],
       [0., 0., 0., 0., 0., 0.]])

In [132]:
Y2

array([[5., 0., 1., 2., 3., 4.],
       [5., 0., 1., 2., 3., 4.],
       [5., 0., 1., 2., 3., 4.],
       [5., 0., 1., 2., 3., 4.],
       [5., 0., 1., 2., 3., 4.]])

In [133]:
Y

array([[0., 1., 2., 3., 4., 5.],
       [0., 1., 2., 3., 4., 5.],
       [0., 1., 2., 3., 4., 5.],
       [0., 1., 2., 3., 4., 5.],
       [0., 1., 2., 3., 4., 5.]])

In [140]:
index = 6
print(XX[index], XX2[index])
print(YY[index], YY2[index])

1.0 0.0
0.0 5.0


In [116]:
X

array([[0., 0., 0., 0., 0., 0.],
       [1., 1., 1., 1., 1., 1.],
       [2., 2., 2., 2., 2., 2.],
       [3., 3., 3., 3., 3., 3.],
       [4., 4., 4., 4., 4., 4.]])

In [2]:
(21/22)+(42/11)+(231/128)

6.577414772727273

In [3]:
(21/22)*(42/11)*(231/128)

6.577414772727273

In [99]:
data = np.array([[1], [2], [3], [4], [5], [6], [7], [8], [9], [10]])
ds = TINN_Dataset(tf.float32,
                  data, 
                  2*data)

In [103]:
class dummy_loss(tu.PDE_Residual):
    def __init__(
        self,
        print_precision=".5f",
    ):
        """ """

        super().__init__(name="dummy_loss", print_precision=print_precision)
        self._trainables_ = ()
        

    @tf.function
    def residual(self, pinn, x): 
        y = pinn.net(x)
        return y, tf.constant([0.0] * y.shape[0], dtype=pinn.dtype), tf.constant([0.0] * y.shape[0], dtype=pinn.dtype)

In [112]:
pinn = tu.NN([1, 1], np.array([1.0]), np.array([21.0]))

In [109]:
model = tu.TINN(pinn, dummy_loss(), L2())

In [106]:
list(ds.batch(10).as_numpy_iterator())

[(array([[ 1.],
         [ 2.],
         [ 3.],
         [ 4.],
         [ 5.],
         [ 6.],
         [ 7.],
         [ 8.],
         [ 9.],
         [10.]], dtype=float32),
  array([[ 2.],
         [ 4.],
         [ 6.],
         [ 8.],
         [10.],
         [12.],
         [14.],
         [16.],
         [18.],
         [20.]], dtype=float32))]

In [25]:
pinn(np.array([[.5], [.25]]))

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[0.32972032],
       [0.34861135]], dtype=float32)>

In [122]:
pinn = tu.NN([1, 10, 1], np.array([-10.0]), np.array([10.0]))

In [121]:
with tf.GradientTape(persistent=True) as tape:
    inputs = tf.Variable([[-10.0], [10.0]])    
    o = pinn(inputs)
print(o.numpy())
print(tape.gradient(o, inputs))

with tf.GradientTape(persistent=True) as tape:
    inputs = tf.Variable([[-1.0], [1.0]])    
    o = pinn(inputs)
print(o.numpy())
print(tape.gradient(o, inputs))


with tf.GradientTape(persistent=True) as tape:
    inputs = tf.Variable([[-100.0], [100.0]])    
    o = pinn(inputs)
print(o.numpy())
print(tape.gradient(o, inputs))

[[-0.28114563]
 [ 0.28114563]]
tf.Tensor(
[[0.0089566]
 [0.0089566]], shape=(2, 1), dtype=float32)
[[ 0.13108774]
 [-0.13108774]]
tf.Tensor(
[[-0.04802988]
 [-0.04802988]], shape=(2, 1), dtype=float32)
[[-0.3360556]
 [ 0.3360556]]
tf.Tensor(
[[2.2066615e-06]
 [2.2066615e-06]], shape=(2, 1), dtype=float32)


In [123]:
with tf.GradientTape(persistent=True) as tape:
    inputs = tf.Variable([[-10.0], [10.0]])    
    o = pinn(inputs)
print(o.numpy())
print(tape.gradient(o, inputs))

with tf.GradientTape(persistent=True) as tape:
    inputs = tf.Variable([[-1.0], [1.0]])    
    o = pinn(inputs)
print(o.numpy())
print(tape.gradient(o, inputs))


with tf.GradientTape(persistent=True) as tape:
    inputs = tf.Variable([[-100.0], [100.0]])    
    o = pinn(inputs)
print(o.numpy())
print(tape.gradient(o, inputs))

[[ 0.18756835]
 [-0.18756835]]
tf.Tensor(
[[-0.01437075]
 [-0.01437075]], shape=(2, 1), dtype=float32)
[[ 0.02124835]
 [-0.02124836]]
tf.Tensor(
[[-0.02119057]
 [-0.02119057]], shape=(2, 1), dtype=float32)
[[ 0.38984135]
 [-0.38984135]]
tf.Tensor(
[[-0.00203375]
 [-0.00203375]], shape=(2, 1), dtype=float32)
